In [1]:
from sklearn.metrics import roc_auc_score

In [57]:
import numpy as np
import scipy.stats as stats

In [58]:
import xgboost as xgb
import catboost as cb

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [61]:
from sklearn.model_selection import KFold
from itertools import product,chain

In [5]:
from sklearn.model_selection import RandomizedSearchCV

In [32]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [33]:
labels = pd.read_csv('labels_training_set.csv')

In [34]:
features_1 = pd.read_csv("features_12.csv")

In [35]:
processed = features_1.set_index('person')

In [36]:
cat_features = []
for i,col in enumerate(processed.columns):
    if processed[col].dtype.name=='object':
        cat_features.append(i)

In [37]:
predict_features = kaggle.set_index('person').join(processed)

In [38]:
training_features = labels.set_index('person').join(processed)
training_labels = training_features['label']
training_features = training_features.drop(['label'],axis=1)

In [39]:
def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []

    for t, v in groups.items():

        cs = frame[v].columns
        vs = frame[v]
        lcs = len(cs)

        for i in range(lcs):
            iv = vs.iloc[:,i].tolist()
            for j in range(i+1, lcs):
                jv = vs.iloc[:,j].tolist()
                if iv == jv:
                    dups.append(cs[i])
                    break

    return dups

In [40]:
duplicate_columns(processed)

['is_new',
 'compras',
 'visitas',
 'cant_prod_vistos',
 'lead_1',
 'busquedas',
 'clicks_en_ads',
 'checkouts',
 'entradas_desde_motor',
 'listar_productos',
 'listar_marca']

## Prediccion (Score: 0.86602)

In [50]:
xtrain,xtest,ytrain,ytest = train_test_split(training_features,training_labels)

In [73]:
params= {'depth': 3, 'iterations': 250, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'border_count': 5, 'thread_count': 4} 

In [76]:
cb_classifier = cb.CatBoostClassifier(**params)

In [77]:
cb_classifier.fit(xtrain,ytrain,cat_features=cat_features,verbose=0)

In [ ]:
cb_classifier.eval_metrics(cb.Pool(xtest,ytest,cat_features=cat_features),'AUC',plot=True)

In [45]:
roc_auc_score(ytest,cb_classifier.predict_proba(xtest)[:,1])

0.8402901824500435

In [23]:
cb_classifier.fit(training_features,training_labels,cat_features=cat_features,verbose=0)

In [27]:
proba_loco=cb_classifier.predict_proba(predict_features)[:,1]

In [28]:
submit = kaggle.set_index('person')
submit['label'] = proba_loco

In [29]:
submit.to_csv('submit.csv')

In [30]:
cb_classifier.save_model('preentrenados/86602catboost')